In [ ]:
"""
2. Fazendo uma relação entre número de contatos e sucesso da campanha quais
são os pontos relevantes a serem observados?

"""

In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV


In [62]:
data = pd.read_csv('data/bank-full.csv',sep = ';')

In [63]:
dict_b = {'no':0,'yes':1}
categorical = ['job', 'marital', 'education',  'contact','poutcome' ]
numerical = ['age','balance', 'duration','campaign','previous','pdays' ]
booleans = ['default', 'housing','loan', 'y' ]

In [64]:
data[booleans] = data[booleans].replace(dict_b)
data = pd.get_dummies(data, columns = categorical)

In [74]:
x_col = ['age', 'default', 'balance', 'housing', 'loan', 
       'duration', 'campaign', 'pdays', 'previous', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'contact_cellular', 'contact_telephone',
       'contact_unknown', 'poutcome_failure', 'poutcome_other',
       'poutcome_success', 'poutcome_unknown']

y_col = ['y']

In [75]:
x_train,x_eval, y_train, y_eval = train_test_split(data[x_col], 
                                                 data[y_col],
                                                 test_size=0.2,
                                                 random_state=58)


In [103]:
model = RandomForestClassifier(n_estimators=25,
                                 max_depth=3, 
                                 random_state=58,
                                 criterion= 'entropy'
                              )
clf = model.fit(x_train,y_train)
clf.score(x_eval, y_eval)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8828928452947031

In [104]:
print(pd.crosstab(np.array(y_eval).reshape(1,-1)[0], 
                  clf.predict(x_eval) ,
                  rownames=['Actual'], 
                  colnames=['Predicted'],
                  margins=True))

Predicted     0  1   All
Actual                  
0          7976  0  7976
1          1059  8  1067
All        9035  8  9043


In [105]:
selector = RFECV(clf, step=1, cv=5, n_jobs=-1)
selector = selector.fit(x_train, y_train)
#feature selection + recursivo = método fácil para evitar colineariedades

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [106]:
print(pd.DataFrame(selector.ranking_, index = x_col, columns=['importance']).sort_values('importance', ascending=True))

                     importance
duration                      1
poutcome_success              1
contact_unknown               2
pdays                         3
housing                       4
previous                      5
contact_cellular              6
age                           7
poutcome_failure              8
poutcome_unknown              9
campaign                     10
marital_married              11
balance                      12
education_tertiary           13
marital_single               14
education_secondary          15
job_student                  16
marital_divorced             17
loan                         18
job_unknown                  19
job_retired                  20
poutcome_other               21
job_self-employed            22
job_unemployed               23
job_blue-collar              24
default                      25
education_primary            26
job_admin.                   27
education_unknown            28
job_technician               29
job_mana

In [ ]:
"""
Opa, conseguiremos otimizar a campanha olhando para a duração, olhando o resultado da 
ultima campanha, a idade e se a pessoa tem empréstimo imobiliário
""""